# Overview

This example demonstrates how to create a bioinformatics workflow using Snakemake. The workflow includes preprocessing raw sequencing data, performing an analysis, and generating a report.

# Installation

In [ ]:
! pip install snakemake
! sudo apt-get install fastqc

# Project Structure Outline

```bash
./
    |- src/
        |- scripts/
            |- preprocessing/
                |- preprocess.py
            |- analysis/
                |- analyze.py
            |- postprocessing/
                |- postprocess.py
            |- utilities/
                |- log.py
                |- report.py
    |- data/
        |- raw/
            |- sample1.fastq
            |- sample2.fastq
        |- processed/
        |- results/
    |- out/
        |- logs/
        |- reports/
        |- figures/
        |- temp/
    |- rules/
        |- preprocess.smk
        |- analyze.smk
        |- postprocess.smk
        |- report.smk
        |- log.smk
        |- temp.smk
    |- sandbox/
    README.md
    Snakefile

```

# Create The Structure

In [1]:
import os

# Define directory names
directories = [
    "src/scripts/preprocessing",
    "src/scripts/analysis",
    "src/scripts/postprocessing",
    "src/scripts/utilities",
    "src/modules",
    "src/config",
    "src/lib",
    "data/raw",
    "data/processed",
    "data/results",
    "out/logs",
    "out/reports",
    "out/figures",
    "out/temp",
    "rules",
    "sandbox"
]

# Create directories
for directory in directories:
    os.makedirs(directory, exist_ok=True)

# Create sample data for sample1.fastq and sample2.fastq
sample1_content = """\
@Sample1_1
ACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTG
+
!""#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNO

@Sample1_2
CAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGT
+
!""#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNO
"""

sample2_content = """\
@Sample2_1
GCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTA
+
!""#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNO

@Sample2_2
TCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGA
+
!""#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNO
"""

# Write sample data to files
with open("data/raw/sample1.fastq", "w") as file:
    file.write(sample1_content)

with open("data/raw/sample2.fastq", "w") as file:
    file.write(sample2_content)

# Create empty files
empty_files = [
    "src/scripts/preprocessing/preprocess.py",
    "src/scripts/analysis/analyze.py",
    "src/scripts/postprocessing/postprocess.py",
    "src/scripts/utilities/log.py",
    "src/scripts/utilities/report.py",
    "rules/preprocess.smk",
    "rules/analyze.smk",
    "rules/postprocess.smk",
    "rules/log.smk",
    "rules/report.smk",
    "rules/temp.smk",
    "README.md",
    "Snakefile"
]

for file_path in empty_files:
    if not os.path.exists(file_path):
        open(file_path, "w").close()

print("Directory structure created successfully.")


Directory structure created successfully.


- Now, that the structure is ready, let's fill out the files with their corresponding codes

# Writing Rules

- ***rules/preprocess.smk***
```python
rule preprocess:
    input:
        "data/raw/{sample}.fastq"
    output:
        "data/processed/{sample}_preprocessed.fastq"
    shell:
        "python src/scripts/preprocessing/preprocess.py {input} {output}"
```

- ***rules/analyze.smk***

```python
rule analyze:
    input:
        "data/processed/{sample}_preprocessed.fastq"
    output:
        "data/results/{sample}_analysis.txt"
    shell:
        "python src/scripts/analysis/analyze.py {input} {output}"
```

- ***rules/postprocess.smk***

```python
rule postprocess:
    input:
        "data/results/{sample}_analysis.txt"
    output:
        "data/results/{sample}_gc_content.txt"
    shell:
        "python src/scripts/postprocessing/postprocess.py {input} {output}"
```

- ***rules/report.smk***
```python
rule report:
    input:
        "data/results/{sample}_gc_content.txt"
    output:
        "out/reports/{sample}_report.txt"
    shell:
        "python src/scripts/utilities/report.py {input} {output}"
```

- ***rules/log.smk***
```python
rule log:
    input:
        "data/raw/{sample}.fastq"
    output:
        "out/logs/{sample}.log"
    shell:
        "python src/scripts/utilities/log.py 'Processing {wildcards.sample}' {output}"
```

- ***rules/temp.smk***
```python
rule temp_file:
    input:
        "data/raw/{sample}.fastq"
    output:
        temp("out/temp/{sample}_temp.txt")
    shell:
        "echo 'Temporary file for {wildcards.sample}' > {output}"
```

- These are the rules that we want to follow in this workflow. You can replace the contets of the mentioned files with these codes.

# Writing Scripts

- ***src/scripts/preprocessing/preprocess.py***

In [4]:
"""
This script reads a FASTQ file, processes its content by converting all characters to uppercase, 
and writes the processed content to a new file.

Usage:
    python preprocess.py input.fastq output.fastq
"""

import sys

def preprocess(input_file, output_file):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            # Example preprocessing: convert to uppercase
            f_out.write(line.upper())

if __name__ == "__main__":
    input_file = sys.argv[1]
    output_file = sys.argv[2]
    preprocess(input_file, output_file)

- ***src/scripts/analysis/analyze.py***

In [ ]:
"""
This script reads a preprocessed FASTQ file and counts the occurrences of each nucleotide ('A', 'T', 'C', 'G').
It writes the counts to an output file.

Usage:
    python analyze.py input_preprocessed.fastq output_analysis.txt
"""

import sys

def analyze(input_file, output_file):
    counts = {'A': 0, 'T': 0, 'C': 0, 'G': 0}
    with open(input_file, 'r') as f_in:
        for line in f_in:
            if not line.startswith('@') and not line.startswith('+'):
                for char in line.strip():
                    if char in counts:
                        counts[char] += 1
    with open(output_file, 'w') as f_out:
        for nucleotide, count in counts.items():
            f_out.write(f"{nucleotide}: {count}\n")

if __name__ == "__main__":
    input_file = sys.argv[1]
    output_file = sys.argv[2]
    analyze(input_file, output_file)

- ***src/scripts/postprocessing/postprocess.py***

In [ ]:
"""
This script reads an analysis file containing nucleotide counts, calculates the GC content,
and writes this information to an output file.

Usage:
    python postprocess.py input_analysis.txt output_gc_content.txt
"""

import sys

def calculate_gc_content(input_file, output_file):
    total_bases = 0
    gc_bases = 0
    with open(input_file, 'r') as f_in:
        for line in f_in:
            if line.startswith('A:') or line.startswith('T:') or line.startswith('C:') or line.startswith('G:'):
                nucleotide, count = line.strip().split(': ')
                count = int(count)
                total_bases += count
                if nucleotide in ['G', 'C']:
                    gc_bases += count
    gc_content = (gc_bases / total_bases) * 100 if total_bases > 0 else 0
    with open(output_file, 'w') as f_out:
        f_out.write(f"GC Content: {gc_content:.2f}%\n")

if __name__ == "__main__":
    input_file = sys.argv[1]
    output_file = sys.argv[2]
    calculate_gc_content(input_file, output_file)

- ***src/scripts/utilities/log.py***

In [ ]:
"""
This script logs a message with a timestamp to an output log file.

Usage:
    python log.py "message" output.log
"""

import sys
import datetime

def log_message(message, output_file):
    with open(output_file, 'a') as f_out:
        f_out.write(f"{datetime.datetime.now()}: {message}\n")

if __name__ == "__main__":
    message = sys.argv[1]
    output_file = sys.argv[2]
    log_message(message, output_file)

- ***src/scripts/utilities/report.py***

In [ ]:
"""
This script generates a report based on the input GC content file and writes it to an output file.

Usage:
    python report.py input_gc_content.txt output_report.txt
"""

import sys

def generate_report(input_file, output_file):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        f_out.write("Report\n")
        f_out.write("======\n\n")
        for line in f_in:
            f_out.write(line)

if __name__ == "__main__":
    input_file = sys.argv[1]
    output_file = sys.argv[2]
    generate_report(input_file, output_file)

# Writing the Snakefile

- ***Snakefile***

```python
include: "rules/preprocess.smk"
include: "rules/analyze.smk"
include: "rules/postprocess.smk"
include: "rules/report.smk"
include: "rules/log.smk"
include: "rules/temp.smk"

rule all:
    input:
        expand("data/results/{sample}_gc_content.txt", sample=["sample1", "sample2"]),
        expand("out/logs/{sample}.log", sample=["sample1", "sample2"]),
        expand("out/reports/{sample}_report.txt", sample=["sample1", "sample2"]),
        expand("out/temp/{sample}_temp.txt", sample=["sample1", "sample2"])
```

# Run the workflow

In [2]:
! snakemake --cores all

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 12
Rules claiming more threads will be scaled down.
Job stats:
job            count
-----------  -------
all                1
analyze            2
log                2
postprocess        2
preprocess         2
report             2
temp_file          2
total             13

Select jobs to execute...

[Sun May 26 08:09:16 2024]
rule temp_file:
    input: data/raw/sample1.fastq
    output: out/temp/sample1_temp.txt
    jobid: 11
    reason: Missing output files: out/temp/sample1_temp.txt
    wildcards: sample=sample1
    resources: tmpdir=/tmp


[Sun May 26 08:09:16 2024]
rule preprocess:
    input: data/raw/sample2.fastq
    output: data/processed/sample2_preprocessed.fastq
    jobid: 6
    reason: Missing output files: data/processed/sample2_preprocessed.fastq
    wildcards: sample=sample2
    resources: tmpdir=/tmp


[Sun May 26 08:09:16 2024]
rule log:
    input: data/raw/sample2.fastq
    output: out/logs/sample2.log


- If you are getting an error like this `AttributeError: module 'pulp' has no attribute 'list_solvers'. Did you mean: 'listSolvers'?`, try downgrading the `pulp` package:

In [ ]:
! pip install PuLP==2.7.0